# Panduan Menjalankan Algoritma Round Robin Menggunakan NGINX untuk Sistem Multi-Agent dan Multi-Client

## 📘 Deskripsi Singkat
Proyek ini mengimplementasikan **algoritma Round Robin Load Balancing** menggunakan **NGINX** untuk mendistribusikan beban permintaan secara bergantian ke dua server backend (Agent-1 dan Agent-2).  
Setiap agent menjalankan aplikasi **Flask sederhana** yang menampilkan halaman web HTML dan endpoint `/process` untuk simulasi pemrosesan data.

Arsitektur ini merepresentasikan **sistem multi-agent dan multi-client pada jaringan lokal (LAN)**.


## ⚙️ Topologi Sistem

| Peran | Perangkat | IP Address | Port | Fungsi Utama |
|-------|------------|-------------|------|---------------|
| **Load Balancer** | Ubuntu Server | `192.168.1.72` | 8000 | Menjalankan NGINX Reverse Proxy |
| **Agent-1** | Windows Laptop | `192.168.1.70` | 5001 | Menyediakan halaman HTML & endpoint `/process` |
| **Agent-2** | Ubuntu Laptop | `192.168.1.72` | 5002 | Menyediakan halaman HTML & endpoint `/process` |

Alur kerja:

```
Client → NGINX (192.168.1.72:8000)
↳ Agent-1 (192.168.1.70:5001)
↳ Agent-2 (192.168.1.72:5002)
```

## 🧠 Tujuan

- Mengimplementasikan **algoritma Round Robin** untuk load balancing.
- Menunjukkan bagaimana **NGINX** dapat mendistribusikan permintaan antar backend server.
- Mengukur **waktu respons rata-rata** dan **deviasi standar** untuk 50, 100, dan 200 request.


## 🧩 Komponen yang Digunakan
- **NGINX 1.24+** — sebagai Load Balancer.  
- **Python 3.12 + Flask** — untuk masing-masing Agent.  
- **Ubuntu 24.04** — untuk Load Balancer dan Agent-2.  
- **Windows 10/11** — untuk Agent-1.  
- **Network lokal (LAN/WiFi)** — untuk komunikasi antar node.


## ⚙️ 1️⃣ Instalasi di Load Balancer (Ubuntu)
### Langkah 1: Instalasi NGINX
```bash
sudo apt update
sudo apt install nginx -y
sudo systemctl status nginx
```
Cek hasil:
Buka `http://192.168.1.72/` → muncul halaman **“Welcome to nginx!”**


### Langkah 2: Konfigurasi Load Balancer
```bash
sudo nano /etc/nginx/sites-available/roundrobin-lb
```
Isi dengan:
```nginx
upstream flask_backends {
    server 192.168.1.70:5001;
    server 192.168.1.72:5002;
}

server {
    listen 8000;
    server_name 192.168.1.72;

    location / {
        proxy_pass http://flask_backends;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
    }

    access_log /var/log/nginx/roundrobin_access.log;
    error_log  /var/log/nginx/roundrobin_error.log;
}
```
Aktifkan konfigurasi:
```bash
sudo ln -s /etc/nginx/sites-available/roundrobin-lb /etc/nginx/sites-enabled/
sudo rm /etc/nginx/sites-enabled/default
sudo nginx -t
sudo systemctl reload nginx
```


## ⚙️ 2️⃣ Instalasi Agent-1 (Windows)
### Folder Struktur
```
C:\Users\<User>\Desktop\roundrobin-agent1\
```
### File `agent1.py`


In [ ]:
from flask import Flask, jsonify, request, render_template
import time, socket

app = Flask(__name__, template_folder="templates")
SERVER_ID = "Agent-1"

@app.route("/")
def home():
    return render_template("agent1.html", server_id=SERVER_ID)

@app.route("/process")
def process():
    start = time.time()
    time.sleep(0.05)
    duration = (time.time() - start) * 1000
    return jsonify({
        "server_id": SERVER_ID,
        "message": f"Request diproses oleh {SERVER_ID}",
        "host": socket.gethostname(),
        "processing_time_ms": duration,
        "client_from_lb": request.remote_addr
    })

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001)

### File `templates/agent1.html`
```html
<!DOCTYPE html>
<html lang="id">
<head><meta charset="UTF-8"><title>{{ server_id }}</title>
<style>
body { font-family: Arial; text-align: center; background: #1e3a8a; color: white; }
.card { margin-top: 100px; padding: 40px; background: rgba(255,255,255,0.1); border-radius: 10px; display: inline-block; }
</style></head>
<body><div class="card">
<h1>{{ server_id }}</h1>
<p>Backend: Windows Agent-1</p>
<p>IP: 192.168.1.70:5001</p></div></body></html>
```
### Jalankan Agent-1
```bat
python -m pip install flask
python agent1.py
```


## ⚙️ 3️⃣ Instalasi Agent-2 (Ubuntu)
### File `agent2.py`


In [ ]:
from flask import Flask, jsonify, request, render_template
import time, socket

app = Flask(__name__, template_folder="templates")
SERVER_ID = "Agent-2-Ubuntu"

@app.route("/")
def home():
    return render_template("agent2.html", server_id=SERVER_ID)

@app.route("/process")
def process():
    start = time.time()
    time.sleep(0.08)
    duration = (time.time() - start) * 1000
    return jsonify({
        "server_id": SERVER_ID,
        "message": f"Request diproses oleh {SERVER_ID}",
        "host": socket.gethostname(),
        "processing_time_ms": duration,
        "client_from_lb": request.remote_addr
    })

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5002)

### File `templates/agent2.html`
```html
<!DOCTYPE html>
<html lang="id">
<head><meta charset="UTF-8"><title>{{ server_id }}</title>
<style>
body { font-family: Arial; text-align: center; background: #14532d; color: white; }
.card { margin-top: 100px; padding: 40px; background: rgba(255,255,255,0.1); border-radius: 10px; display: inline-block; }
</style></head>
<body><div class="card">
<h1>{{ server_id }}</h1>
<p>Backend: Ubuntu Agent-2</p>
<p>IP: 192.168.1.72:5002</p></div></body></html>
```
### Jalankan Agent-2
```bash
(venv) python agent2.py
```


## 🌐 4️⃣ Pengujian Sistem
Buka di browser:
```
http://192.168.1.72:8000/
```
🔁 Refresh beberapa kali:
- Kadang tampil halaman Agent-1 (biru)
- Kadang tampil halaman Agent-2 (hijau)

✅ Menandakan **Round Robin Load Balancing via NGINX** bekerja.


## 🧪 5️⃣ Uji Beban (Opsional)
### File `client_load_test.py`


In [ ]:
import requests, time, statistics

LB_URL = "http://192.168.1.72:8000/process"

def run_scenario(total_requests):
    print(f"\n=== Uji Beban {total_requests} Request (via NGINX) ===")
    times, backend_counts = [], {}
    for i in range(1, total_requests + 1):
        start = time.time()
        resp = requests.get(LB_URL)
        elapsed = (time.time() - start) * 1000
        try:
            data = resp.json()
            backend = data.get("server_id", "Unknown")
        except:
            backend = "Unknown"
        times.append(elapsed)
        backend_counts[backend] = backend_counts.get(backend, 0) + 1
        print(f"Req-{i:03d} | {elapsed:7.2f} ms | Backend: {backend}")
    avg = sum(times)/len(times)
    std = statistics.stdev(times) if len(times)>1 else 0.0
    print(f"\nRata-rata: {avg:.2f} ms | Deviasi: {std:.2f} ms")
    print("Distribusi Backend:")
    for b,c in backend_counts.items():
        print(f"  - {b}: {c} request")

if __name__=="__main__":
    for n in [50,100,200]:
        run_scenario(n)
        print("\n"+"="*40+"\n")

## 🧩 6️⃣ Panduan Menjalankan Sistem

1️⃣ Jalankan `agent1.py` di Windows.  
2️⃣ Jalankan `agent2.py` di Ubuntu.  
3️⃣ Reload NGINX:
```bash
sudo systemctl reload nginx
```
4️⃣ Akses browser `http://192.168.1.72:8000/`  
5️⃣ Refresh → web akan bergantian (Round Robin).


## 🧭 7️⃣ Cara Membedakan Flask LB dan NGINX LB

| Ciri | Flask LB Aktif | NGINX LB Aktif |
|------|----------------|----------------|
| Port 8000 dimiliki oleh | `python3` | `nginx` |
| Header HTTP | `Server: Werkzeug` | `Server: nginx` |
| Log | tampil di terminal Python | `/var/log/nginx/roundrobin_access.log` |
| Perlu `python load_balancer.py` | ✅ Ya | ❌ Tidak perlu |
| Jalankan via `systemctl` | ❌ Tidak | ✅ Ya |

**Cek cepat di terminal:**  
```bash
sudo lsof -i :8000
systemctl status nginx
curl -I http://192.168.1.72:8000/
```
Jika `Server: nginx` → berarti benar-benar pakai NGINX.


## 👨‍💻 Penulis
**Wiwi Nopiana**  
Implementasi Algoritma Round Robin dalam Sistem Multi-Agent dan Multi-Client untuk Load Balancing Dinamis pada Jaringan Lokal  
Supervised by **Muh. Fuad Mansyur** & **Wawan Firgiawan**
